In [5]:
import pandas as pd
transaction_data = pd.read_csv("transaction_data.csv")
purchase_behaviour = pd.read_csv("purchase_behaviour.csv")

In [7]:
# print(transaction_data.info())
print(transaction_data.head())

    DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  43390          1            1000       1         5   
1  43599          1            1307     348        66   
2  43605          1            1343     383        61   
3  43329          2            2373     974        69   
4  43330          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  
0    Natural Chip        Compny SeaSalt175g         2        6.0  
1                  CCs Nacho Cheese    175g         3        6.3  
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9  
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0  
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8  


In [8]:
# print(purchase_behaviour.info())
print(purchase_behaviour.head())

   LYLTY_CARD_NBR               LIFESTAGE PREMIUM_CUSTOMER
0            1000   YOUNG SINGLES/COUPLES          Premium
1            1002   YOUNG SINGLES/COUPLES       Mainstream
2            1003          YOUNG FAMILIES           Budget
3            1004   OLDER SINGLES/COUPLES       Mainstream
4            1005  MIDAGE SINGLES/COUPLES       Mainstream


In [9]:
# Compute total sales per product
top_products = transaction_data.groupby("PROD_NAME")["TOT_SALES"].sum().nlargest(3)
print("Top 3 Most Profitable Products:\n", top_products)

Top 3 Most Profitable Products:
 PROD_NAME
Dorito Corn Chp     Supreme 380g            40352.0
Smiths Crnkle Chip  Orgnl Big Bag 380g      36367.6
Smiths Crinkle Chips Salt & Vinegar 330g    34804.2
Name: TOT_SALES, dtype: float64


In [15]:
# Find the most frequent customer
loyal_customers = transaction_data["LYLTY_CARD_NBR"].value_counts().nlargest(10).index
print("Top 10 Most Loyal Customers:\n", loyal_customers)

Top 10 Most Loyal Customers:
 Index([172032, 162039, 230078, 128178, 13138, 116181, 222242, 94185, 113080,
       109036],
      dtype='int64', name='LYLTY_CARD_NBR')


In [16]:
# Merge transaction and customer data
merged_data = transaction_data.merge(purchase_behaviour, on="LYLTY_CARD_NBR", how="left")
print(merged_data.head())

    DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  43390          1            1000       1         5   
1  43599          1            1307     348        66   
2  43605          1            1343     383        61   
3  43329          2            2373     974        69   
4  43330          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  \
0    Natural Chip        Compny SeaSalt175g         2        6.0   
1                  CCs Nacho Cheese    175g         3        6.3   
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9   
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0   
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8   

                LIFESTAGE PREMIUM_CUSTOMER  
0   YOUNG SINGLES/COUPLES          Premium  
1  MIDAGE SINGLES/COUPLES           Budget  
2  MIDAGE SINGLES/COUPLES           Budget  
3  MIDAGE SINGLES/COUPLES           Budget  
4  MIDAGE SINGLES/COUPL

In [17]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [18]:
# Aggregate customer spending and transactions
customer_stats = merged_data.groupby("LYLTY_CARD_NBR").agg(
    total_spent=("TOT_SALES", "sum"),
    total_transactions=("TXN_ID", "count")
).reset_index()

In [19]:
scaler = StandardScaler()
customer_stats_scaled = scaler.fit_transform(customer_stats[["total_spent", "total_transactions"]])

In [20]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
customer_stats["cluster"] = kmeans.fit_predict(customer_stats_scaled)
print("Customer Clusters:\n", customer_stats.head())

Customer Clusters:
    LYLTY_CARD_NBR  total_spent  total_transactions  cluster
0            1000          6.0                   1        2
1            1002          2.7                   1        2
2            1003          6.6                   2        2
3            1004          1.9                   1        2
4            1005          2.8                   1        2


In [21]:
# Merge clusters back to customer demographics
clustered_customers = customer_stats.merge(purchase_behaviour, on="LYLTY_CARD_NBR", how="left")

# Summarize clusters
cluster_summary = clustered_customers.groupby(["cluster", "LIFESTAGE", "PREMIUM_CUSTOMER"]).size().reset_index(name="count")
print("Customer Cluster Summary:\n", cluster_summary)

Customer Cluster Summary:
     cluster               LIFESTAGE PREMIUM_CUSTOMER  count
0         0  MIDAGE SINGLES/COUPLES           Budget    545
1         0  MIDAGE SINGLES/COUPLES       Mainstream   1253
2         0  MIDAGE SINGLES/COUPLES          Premium    857
3         0            NEW FAMILIES           Budget    390
4         0            NEW FAMILIES       Mainstream    314
..      ...                     ...              ...    ...
58        2          YOUNG FAMILIES       Mainstream   1090
59        2          YOUNG FAMILIES          Premium   1000
60        2   YOUNG SINGLES/COUPLES           Budget   2799
61        2   YOUNG SINGLES/COUPLES       Mainstream   5465
62        2   YOUNG SINGLES/COUPLES          Premium   1882

[63 rows x 4 columns]
